In [10]:
import os
import numpy as np
import librosa
import torchaudio

In [11]:
# import matplotlib.pyplot as plt

# path = '../vakyansh-wav2vec2-experimentation/data/pretraining/data_thchs30/data/A2_0.wav'
# w1,s1 = torchaudio.load(path)
# w2,s2 = librosa.load(path, sr=16000)
# print(w1,w1.shape,s1)
# print(w2,w2.shape,s2)

# mel1 = librosa.feature.melspectrogram(y=w1.squeeze(0).numpy(), sr=16000, n_fft=640, n_mels=128,
#                                                         hop_length=320, win_length=640)
# mel2 = librosa.feature.melspectrogram(y=w2, sr=16000, n_fft=640, n_mels=128,
#                                                         hop_length=320, win_length=640)
# print(np.all(mel1==mel2))

# # mel = mel.transpose(1, 0)
# # print(mel)
# # mel = librosa.power_to_db(mel)
# # print(mel)
# # mel = librosa.util.normalize(mel)
# # print(mel)

# # plt.imshow(mel)
# # plt.show()

In [ ]:
os.listdir('../vakyansh-wav2vec2-experimentation/data/pretraining/data_thchs30')

In [12]:
folder = '../vakyansh-wav2vec2-experimentation/data/pretraining/data_thchs30'
lm_phone_folder, lm_phone_file = 'lm_phone', 'lexicon.txt'
train_folder, valid_folder, test_folder, data_folder = 'train', 'dev', 'test', 'data'

In [13]:
file = os.path.join(folder, lm_phone_folder, lm_phone_file)
VOCAB = []
with open(file, 'r') as f:
    lines = f.readlines()
    for line in lines:
        phone = line.split()[0]
        VOCAB.append(phone)
print(len(VOCAB))

219


In [14]:
def str2token(s, vocab):
    return vocab.index(s)

In [15]:
win_length = 640
hop_length = 320
n_mels = 128

In [ ]:
wav_folder = test_folder
files = os.listdir(os.path.join(folder, wav_folder))
inputs = []
labels = []

from model import Wav2Vec2Finetuner
model = Wav2Vec2Finetuner(None, 219)
inputs_length = []
for file in files:
    # For every .wav, get name
    if file.endswith('wav'):
        name = file.split('.')[0]
        
        wav_path = os.path.join(folder, wav_folder, file)
        with open(wav_path, 'r') as f:
            # waveform: tensor (1, frames)
            waveform, sample_rate = torchaudio.load(wav_path)
            inputs.append(waveform)
            
            out, _ = model(waveform, 0)
            inputs_length.append(out.shape[1])
            
            # mel: ndarray (1, n_wins, n_mels)
            # mel = librosa.feature.melspectrogram(y=waveform.numpy(), sr=sample_rate, n_fft=win_length, n_mels=n_mels,
            #                                     hop_length=hop_length, win_length=win_length)
            # mel = mel.transpose(0, 2, 1)
            # mel = librosa.power_to_db(mel)
            # mel = librosa.util.normalize(mel)
            # inputs.append(mel)
        
        label_path = os.path.join(folder, data_folder, f"{name}.wav.trn")
        with open(label_path, 'r') as f:
            # Need <eps> explicitly or not?
            # labels: ndarray (n_labels)
            label_seq = np.array([str2token(s, VOCAB) for s in f.readlines()[2].strip().split()], dtype=int)
            labels.append(label_seq)

# inputs_length = []
# # max_inputs_length = max([i.shape[1] for i in inputs])
# # print(max_inputs_length)
# for i, ipt in enumerate(inputs):
#     inputs_length.append(ipt.shape[1])
#     # to_pad = max_inputs_length - ipt.shape[1]
#     # inputs[i] = np.pad(ipt, ((0,0),(0,to_pad)), mode='constant', constant_values=0)

labels_length = []
# max_labels_length = max([len(l) for l in labels])
# print(max_labels_length)
for i, lbl in enumerate(labels):
    labels_length.append(lbl.shape[0])
    # to_pad = max_labels_length - lbl.shape[0]
    # labels[i] = np.expand_dims(np.pad(lbl, ((0,to_pad)), mode='constant', constant_values=0), 0)

print(len(labels))

for i in range(len(labels)):
    print(inputs_length[i])
    np.savez(f"./Dataset/raw/test/{i:>04}.npz", input=inputs[i].squeeze().numpy(), input_length=inputs_length[i], label=labels[i], label_length=labels_length[i])

In [26]:

for i in range(len(labels)):
    np.savez(f"./Dataset/raw/train/{i:>04}.npz", input=inputs[i].squeeze().numpy(), input_length=inputs_length[i], label=labels[i], label_length=labels_length[i])

In [19]:
print(len(inputs), inputs[0].shape, inputs[-1].shape)
print(len(inputs_length), inputs_length[0])
print(len(labels), labels[0].shape, labels[-1].shape)
print(len(labels_length), labels_length[0])

10000 (1, 250000) (1, 250000)
10000 124800
10000 (1, 94) (1, 94)
10000 60


In [20]:
inputs = np.concatenate(inputs)
inputs_length = np.array(inputs_length, dtype=int)
labels = np.concatenate(labels)
labels_length = np.array(labels_length, dtype=int)

In [149]:
# print(len(inputs), inputs[0].shape, np.concatenate(inputs).shape)
# print(np.array(inputs_length, dtype=int))
# print(len(labels), labels[0].shape, np.concatenate(labels).shape)
# print(np.array(labels_length, dtype=int))

In [21]:
np.savez('./Dataset/raw/train.npz', inputs=inputs, inputs_length=inputs_length, labels=labels, labels_length=labels_length)

In [22]:
np.save('./Dataset/raw/VOCAB.npy', VOCAB)

In [23]:
ret = np.load('./Dataset/raw/train.npz')
print(ret['inputs'].shape)
print(ret['inputs_length'].shape)
print(ret['labels'].shape)
print(ret['labels_length'].shape)
# np.load('./Dataset/VOCAB.npy').tolist()

(10000, 250000)
(10000,)
(10000, 94)
(10000,)


In [159]:
split = 'train'
data = np.load(f'./Dataset/raw/{split}.npz')
inputs = data['inputs']
inputs_length = data['inputs_length']
labels = data['labels']
labels_length = data['labels_length']
for i in range(len(labels)):
    np.savez(f"./Dataset/{split}/{i:>04}.npz", input=inputs[i], input_length=inputs_length[i], label=labels[i], label_length=labels_length[i])

In [160]:
with open('./Dataset/valid/len.txt', 'r') as f:
    len = int(f.readline())
print(len)

893


In [166]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.plugins.io import TorchCheckpointIO as tcio
from model import CTCEncoderDecoder
from plmodel import LightningCTC

config = {
        'n_mels': 128,
        'n_fft': 640,
        'win_length': 640,
        'hop_length': 320,
        'wav_max_length': 782,
        'transcript_max_length': 94,
        'learning_rate': 1e-3,
        'batch_size': 64,
        'weight_decay': 0,
        'encoder_num_layers': 4,
        'encoder_hidden_dim': 256,
        'encoder_bidirectional': True,
    }

tc = tcio()
ckpt_dict = tc.load_checkpoint('./Checkpoints/LSTM_test/epoch=77-step=12168.ckpt')
print(ckpt_dict.keys())
plmodel = LightningCTC(**config)
plmodel.load_state_dict(ckpt_dict['state_dict'])
model = plmodel.model

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])
